In [1]:
import numpy as np
from scipy.linalg import eigh
import matplotlib.pyplot as plt
import math
import pandas as pd
import sympy as sp


In [1]:
import numpy as np

def build_torsional_system(I_list, k_list):
    """
    Builds the mass (inertia) matrix and stiffness matrix for 
    an n-inertia torsional system in a row.

    Parameters:
    -----------
    I_list : list of floats
        Inertias [I1, I2, ..., In].
    k_list : list of floats
        Springs [k1, k2, ..., kn].
        k1 -> between inertia 1 and ground
        k2 -> between inertia 1 and inertia 2
        ...
        kn -> between inertia n and ground (or boundary)

    Returns:
    --------
    I_mat : (n x n) numpy array
        Diagonal matrix of inertias
    K_mat : (n x n) numpy array
        Torsional stiffness matrix
    """

    n = len(I_list)  # number of inertias
    # Build inertia matrix (diagonal)
    I_mat = np.diag(I_list)

    # Build stiffness matrix
    K_mat = np.zeros((n, n))

    # Fill the diagonal entries
    for i in range(n):
        # If i == 0, the diagonal gets k1 + k2
        # If i == n-1, the diagonal gets k_{n-1} + k_n 
        # Otherwise, k_{i} + k_{i+1}
        # But we must check array indexing carefully.

        # Add spring from ground to inertia i (k[i]) if it exists
        # i.e. if i == 0, we add k[0] to diagonal
        # if i == n-1, we add k[n-1] (the last one) 
        # or generally k[i] if that spring is "to ground."
        # This logic may differ depending on your exact layout.

        # For simplicity, let's assume each inertia has a "to ground" spring k[i]
        # plus a "to next inertia" spring k[i+1], except for the last one.
        # Adjust according to your exact physical configuration.

        # Always add the spring to ground (k[i]) if i < len(k_list):
        if i < len(k_list):
            K_mat[i, i] += k_list[i]

        # Add the spring to the next inertia, if i+1 < n:
        if i+1 < n and i+1 < len(k_list):
            K_mat[i, i] += k_list[i+1]

    # Fill the off-diagonal entries
    for i in range(n-1):
        # The spring between inertia i and inertia i+1 is k[i+1]
        # (assuming that is how you arranged the k_list).
        # Put it in -k[i+1] in (i, i+1) and (i+1, i).
        if i+1 < len(k_list):
            K_mat[i,   i+1] = -k_list[i+1]
            K_mat[i+1, i  ] = -k_list[i+1]

    return I_mat, K_mat

# -----------
# Example usage
# -----------
if __name__ == "__main__":
    # Suppose we have 3 inertias and 3 springs
    I_list = [10.0, 12.0, 5.0]  # e.g. [I1, I2, I3]
    k_list = [1000.0, 2000.0, 1500.0]  # e.g. [k1, k2, k3]
    I_mat, K_mat = build_torsional_system(I_list, k_list)
    print("Inertia matrix I_mat:\n", I_mat)
    print("Stiffness matrix K_mat:\n", K_mat)


Inertia matrix I_mat:
 [[10.  0.  0.]
 [ 0. 12.  0.]
 [ 0.  0.  5.]]
Stiffness matrix K_mat:
 [[ 3000. -2000.     0.]
 [-2000.  3500. -1500.]
 [    0. -1500.  1500.]]
